In [20]:
from keras.models import load_model
import numpy as np
import pickle

data_height = 6000
data_width = 87
label_team_width = 3
label_player_width = 29
label_pass_width = 29

def normalize(data):
    with open('../model/mean_std.pkl', 'rb') as file:
        mean, std = pickle.load(file)
    return (data - mean) / std

Test match array

In [21]:
first_test_match_id = 60566
test_match_count = 1

x_test = np.ndarray(shape=(test_match_count, data_height, data_width), dtype=np.float)
y_test_team = np.ndarray(shape=(test_match_count, data_height, label_team_width), dtype=np.int)
y_test_player = np.ndarray(shape=(test_match_count, data_height, label_player_width), dtype=np.int)
y_test_pass = np.ndarray(shape=(test_match_count, data_height, label_pass_width), dtype=np.int)

for id in range(test_match_count):
    x_test[id] = np.load('../data/match_' + str(id + first_test_match_id) + '/x_data.npy')
    y_test_team[id] = np.load('../data/match_' + str(id + first_test_match_id) + '/y_team_data.npy')
    y_test_player[id] = np.load('../data/match_' + str(id + first_test_match_id) + '/y_player_data.npy')
    y_test_pass[id] = np.load('../data/match_' + str(id + first_test_match_id) + '/y_pass_data.npy')

In [22]:
x_test = normalize(x_test)

Team prediction

In [23]:
team_model = load_model('../model/team_model.h5')

In [24]:
score = team_model.evaluate(x_test, y_test_team)
print('team possesion test acc: ', score[1])

1/1 [==============================] - 2s 2s/step


team possesion test acc:  0.4440000057220459


In [25]:
y_team_pred = team_model.predict(x_test)

for id in range(test_match_count):
    with open('../data/match_' + str(id + first_test_match_id) + '/y_team_pred.npy', 'wb') as file:
        np.save(file, y_team_pred)

Player prediction

In [ ]:
player_model = load_model('../model/player_model.h5')

In [ ]:
x_test = np.concatenate((x_test, y_team_pred), axis=2)

In [ ]:
score = player_model.evaluate(x_test, y_test_player)
print('player possesion test acc: ', score[1])

In [ ]:
y_player_pred = player_model.predict(x_test)

for id in range(test_match_count):
    with open('../data/match_' + str(id + first_test_match_id) + '/y_player_pred.npy', 'wb') as file:
        np.save(file, y_player_pred)

Pass prediction

In [ ]:
pass_model = load_model('../model/pass_model.h5')

In [ ]:
x_test = np.concatenate((x_test, y_player_pred), axis=2)

In [ ]:
score = pass_model.evaluate(x_test, y_test_pas)
print('pass possesion test acc: ', score[1])

In [ ]:
y_pass_pred = pass_model.predict(x_test)

for id in range(test_match_count):
    with open('../data/match_' + str(id + first_test_match_id) + '/y_pass_pred.npy', 'wb') as file:
        np.save(file, y_pass_pred)